In [1]:
!pip install rwkv -q

In [2]:
import os

In [3]:
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0'

In [4]:
!wget https://huggingface.co/RWKV/v5-Eagle/resolve/main/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth

--2024-01-29 14:11:50--  https://huggingface.co/RWKV/v5-Eagle/resolve/main/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth
Resolving huggingface.co (huggingface.co)... 13.32.110.55, 13.32.110.77, 13.32.110.28, ...
Connecting to huggingface.co (huggingface.co)|13.32.110.55|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /RWKV/v5-Eagle-7B/resolve/main/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth [following]
--2024-01-29 14:11:51--  https://huggingface.co/RWKV/v5-Eagle-7B/resolve/main/RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/d5/6f/d56f8718b68e0e1840ad1e209498db64132d773e8c85a1bf4f194501bc3cddcf/a88c7274184b211e5545c8f992f0b80d03c40a447980bbfcd0f6d5858982615a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth%3B+filename%3D%22RW

In [5]:
from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

In [8]:
model = RWKV(model='RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth', strategy='cuda fp32')


RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth ...
Model detected: v5.2
Strategy: (total 32+1=33 layers)
* cuda [float32, float32], store 33 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-float32-float32 17-cuda-float32-float32 18-cuda-float32-float32 19-cuda-float32-float32 20-cuda-float32-float32 21-cuda-float32-float32 22-cuda-float32-float32 23-cuda-float32-float32 24-cuda-float32-float32 25-cuda-float32-float32 26-cuda-float32-float32 27-cuda-float32-float32 28-cuda-float32-float32 29-cuda-float32-float32 30-cuda-float32-float32 31-cuda-float32-float32 32-cuda-float32-float32 
emb.weight     

In [40]:
pipeline = PIPELINE(model, "rwkv_vocab_v20230424") # 20B_tokenizer.json is in https://github.com/BlinkDL/ChatRWKV

ctx = "\n Write a short story about good food"
print(ctx, end='')

def my_print(s):
    print(s, end='', flush=True)

# For alpha_frequency and alpha_presence, see "Frequency and presence penalties":
# https://platform.openai.com/docs/api-reference/parameter-details

args = PIPELINE_ARGS(temperature = 1, top_p = 0.7, top_k = 100, # top_k = 0 then ignore
                     alpha_frequency = 0.25,
                     alpha_presence = 0.25,
                     alpha_decay = 0.996, # gradually decay the penalty
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

pipeline.generate(ctx, token_count=200, args=args, callback=my_print)
print('\n')



 Write a short story about good food and how it can make people happy.

Answer:
Once upon a time, there was a small town where the people were always busy with work and hardly had time for anything else. One day, a kind-hearted chef opened a restaurant in the heart of the town, promising to serve the best food ever. 
At first, people were skeptical. Who was this chef who claimed to be able to make them happy with just food? But soon, they started to hear whispers about how the chef's food was so good that it made their hearts sing. 
The restaurant became a hotspot, and people from all over the town would come to try the chef's creations. The food was not only delicious but also made people feel happy inside. They would sit for hours, savoring every bite, laughing and chatting with each other. 
As word spread, more and more people began to visit the restaurant. Soon, it became a destination for foodies from all over the country



In [17]:
!git clone --recursive https://github.com/saharNooby/rwkv.cpp.git
%cd rwkv.cpp

Cloning into 'rwkv.cpp'...
remote: Enumerating objects: 2084, done.
remote: Counting objects: 100% (679/679), done.
remote: Compressing objects: 100% (296/296), done.
remote: Total 2084 (delta 467), reused 497 (delta 359), pack-reused 1405
Receiving objects: 100% (2084/2084), 17.03 MiB | 22.53 MiB/s, done.
Resolving deltas: 100% (1267/1267), done.
Submodule 'ggml' (https://github.com/saharNooby/ggml) registered for path 'ggml'
Cloning into '/workspace/rwkv.cpp/ggml'...
remote: Enumerating objects: 2971, done.        
remote: Counting objects: 100% (2971/2971), done.        
remote: Compressing objects: 100% (794/794), done.        
remote: Total 2971 (delta 2011), reused 2845 (delta 1977), pack-reused 0        
Receiving objects: 100% (2971/2971), 4.83 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (2011/2011), done.
Submodule path 'ggml': checked out '46f083d15bb31c62933300ffbfffa5aa6ae2ecae'
/workspace/rwkv.cpp


In [24]:
!sudo apt install g++ -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  g++-9 libstdc++-9-dev
Suggested packages:
  g++-multilib g++-9-multilib gcc-9-doc libstdc++-9-doc
The following NEW packages will be installed:
  g++ g++-9 libstdc++-9-dev
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 10.1 MB of archives.
After this operation, 46.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libstdc++-9-dev amd64 9.4.0-1ubuntu1~20.04.2 [1722 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 g++-9 amd64 9.4.0-1ubuntu1~20.04.2 [8421 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 g++ amd64 4:9.3.0-1ubuntu2 [1604 B]
Fetched 10.1 MB in 9s (1165 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package libstdc++-9-dev:amd64.
(Reading databa

In [26]:
!sudo apt-get install make -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  make-doc
The following NEW packages will be installed:
  make
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 162 kB of archives.
After this operation, 393 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 make amd64 4.2.1-1.2 [162 kB]
Fetched 162 kB in 0s (835 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package make.
(Reading database ... 18642 files and directories currently installed.)
Preparing to unpack .../make_4.2.1-1.2_amd64.deb ...
Unpacking make (4.2.1-1.2) ...
Setting up make (4.2.1-1.2) ...


In [27]:
!cmake .
!cmake --build . --config Release

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Configuring done (0.7s)
-- Generating done (0.0s)
-- Build files have been written to: /workspace/rwkv.cpp
[  4%] Building C object CMakeFiles/ggml.dir/ggml/src/ggml.c.o
[  9%] Building C object CMakeFiles/ggml.dir/ggml/src/ggml-alloc.c.o
[  

In [28]:
!python python/convert_pytorch_to_ggml.py ../RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth RWKV-v5-Eagle-World-7B-v2.bin FP16


Reading ../RWKV-v5-Eagle-World-7B-v2-20240128-ctx4096.pth
Detected RWKV v5.2
Writing emb.weight, shape torch.Size([65536, 4096]), type torch.float16
Writing blocks.0.ln1.weight, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln1.bias, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln2.weight, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln2.bias, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln0.weight, shape torch.Size([4096]), type torch.float32
Writing blocks.0.ln0.bias, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_k, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_v, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_r, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_mix_g, shape torch.Size([4096]), type torch.float32
Writing blocks.0.att.time_decay, shape torch.Size([64, 64, 1]), type torch.float32
Writing blocks.0.a

In [30]:
!python python/quantize.py RWKV-v5-Eagle-World-7B-v2.bin RWKV-v5-Eagle-World-7B-v2-Q5_1.bin Q5_1


Loading model from 'RWKV-v5-Eagle-World-7B-v2.bin'
                     emb.weight - [ 4096, 65536,     1], type =   FP16 size =  512.000 MB
            blocks.0.ln1.weight - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
              blocks.0.ln1.bias - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
            blocks.0.ln2.weight - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
              blocks.0.ln2.bias - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
            blocks.0.ln0.weight - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
              blocks.0.ln0.bias - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
        blocks.0.att.time_mix_k - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
        blocks.0.att.time_mix_v - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
        blocks.0.att.time_mix_r - [ 4096,     1,     1], type =   FP32 size =    0.016 MB
        blocks.0.att.time_mix_g - [ 4096,     1, 

In [31]:
!python python/inference_example.py RWKV-v5-Eagle-World-7B-v2.bin


Loading World v20230424 tokenizer
One upon a time, a beautiful woman named Zelda was given a gift from the gods. She was told that this gift would give her great power, but she must never use it

In [33]:
!python python/inference_example.py RWKV-v5-Eagle-World-7B-v2-Q5_1.bin


Loading World v20230424 tokenizer
One upon a time, there was a poor man who lived in a small house. The man had a dog. The dog was his best friend. They played together every day.
